In [1]:
# start up again with our simple example
from databaker.framework import *
tabs = loadxlstabs("DDexample1.xls")
tab = tabs[0]  # there is only one table
#savepreviewhtml(tabs[0])


Loading DDexample1.xls which has size 7168 bytes
Table names: ['beatles', 'stones']


In [2]:
r1 = tab.excel_ref('B3').expand(RIGHT)
r2 = tab.excel_ref('A3').fill(DOWN)
obs = tab.excel_ref('B4').expand(DOWN).expand(RIGHT).is_not_blank().is_not_whitespace()

h1 = HDim(r1, "Vehicles", DIRECTLY, ABOVE) 
b4cell = tab.excel_ref('B4')._cell
print(type(h1))
h1.cellvalueoverride[b4cell] = "Clouds"

dimensions = [ 
    HDim(tab.excel_ref('B1'), TIME, CLOSEST, ABOVE), 
    h1, 
    HDim(r2, "Name", DIRECTLY, LEFT), 
    HDimConst("Category", "Beatles")
]
c1 = ConversionSegment(tab, dimensions, obs)
c1.process()
# make a second conversion segment??

<class 'databaker.jupybakeutils.HDim'>


"TIMEUNIT='Year'"

In [3]:
CompareConversionSegments(c1, "DDex.csv", True)

Conversion seg 0 error: Unmatched new values in segment {('2.0', 'John', 'Clouds', '2014')}
Conversion seg 0 error: Unmatched extra values in wda file {('2.0', 'John', 'Cars', '2014')}


{0: [('NEWVALUESINSEGMENT', {('2.0', 'John', 'Clouds', '2014')}),
  ('WDAEXTRAVALUES', {('2.0', 'John', 'Cars', '2014')})]}

In [4]:
#c1.processedrows[3]

In [5]:
x, Lx = writetechnicalCSV(None, c1), LwritetechnicalCSV(None, c1)
x == Lx, len(x), len(Lx)

(False, 2745, 2757)

In [7]:
def CompareConversionSegments(conversionsegments, wdafile):
    if type(conversionsegments) is ConversionSegment:
        conversionsegments = [conversionsegments]
    
    msglistperseg = { }
    wdasegs = readtechnicalCSV(wdafile)
    extracsegs = [ c  for c in wdasegs.keys()  if not 0<=c<len(conversionsegments) ]
    if extracsegs:
        msglistperseg[-1] = [ ("EXTRAWDACONVERSIONSEGMENTS", extracsegs) ]
    for isegmentnumber, conversionsegment in enumerate(conversionsegments):
        msglist = [ ]
        wdaseg = wdasegs[isegmentnumber]
        headers = extraheaderscheck(c1, wdaseg, msglist)
        headers = checktheconstantdimensions(c1, headers, wdaseg, msglist)
        checksegmentobsvalues(c1.processedrows, headers, wdaseg, msglist)
        if msglist:
            msglistperseg[isegmentnumber] = msglist
    return msglistperseg

msglistperseg = CompareConversionSegments(c1, Lx)
for isegmentnumber, msglist in msglistperseg.items():
    for msg, v in msglist:
        warnings.warn("Conversion seg %d error: " % (isegmentnumber, (wdamsgstrings[msg] % v)))


{}

In [9]:
LwritetechnicalCSV("DDex.csv", [c1])

writing 1 conversion segments into /home/goatchurch/sensiblecode/quickcode-ons-recipes/helpnotes/DDex.csv
conversionwrite segment size 12 table 'beatles; 


In [8]:
import warnings
import collections

# separated out so we can decide the severity of them before printing them out
wdamsgstrings = { 
    "WDAHEADERSINCONSISTENT": "Inconsistent extra headings headings in segment: %s", 
    "WDAHEADERSMISSING": "Extra headings in segment than in wda file: %s", 
    "WDAHEADERSEXTRA": "Extra headings in wda file not in segment: %s", 
    "WDACOLUMNNOTCONSTANT": "Constant column %s has multiple values: %s", 
    "WDACOLUMNCONSTCHANGED": "Constant column %s value changed %s but was in wda file %s", 
    "NEWVALUESINSEGMENT": "Unmatched new values in segment %s", 
    "WDAEXTRAVALUES": "Unmatched extra values in wda file %s", 
    "WDADUPLICATESMISMATCH": "Duplicates mismatche counts %s", 
}

def headersfromwdasegment(wdaseg, msglist):
    derivedheaders = [ OBS ] + (template.SH_Create_ONS_time and [ TIMEUNIT ] or []) + (DATAMARKER and [DATAMARKER] or [])
    headersunion = None
    headersintersection = None
    for wdarow in wdaseg:
        ahset = set(k  for k in wdarow.keys()  if k not in derivedheaders)
        if headersunion is None:
            headersintersection = set(ahset)
            headersunion = set(ahset)
        else:
            headersunion.update(ahset)
            headersintersection.intersection_update(ahset)
    if headersunion != headersintersection:
        msglist.append(("WDAHEADERSINCONSISTENT", headersunion.difference(headersintersection)))
    return headersintersection

def extraheaderscheck(conversionsegment, wdaseg, msglist):
    wdaheaders = headersfromwdasegment(wdaseg, msglist)
    segmentheaders = set([c.label  for c in conversionsegment.dimensions])
    extraheadersinsegment = segmentheaders.difference(wdaheaders)
    extraheadersinwdaseg = wdaheaders.difference(segmentheaders)
    if extraheadersinsegment:
        msglist.append(("WDAHEADERSMISSING", extraheadersinsegment))
    if extraheadersinwdaseg:
        msglist.append(("WDAHEADERSEXTRA", extraheadersinwdaseg))
    return wdaheaders.intersection(segmentheaders)

def checktheconstantdimensions(conversionsegment, headers, wdaseg, msglist):
    for dimension in conversionsegment.dimensions:
        if dimension.label in headers:
            if dimension.hbagset is None:
                constval = dimension.cellvalueoverride.get(None)
                wdaconst = set(row.get(dimension.label)  for row in wdaseg)
                if len(wdaconst) != 1:
                    msglist.append(("WDACOLUMNNOTCONSTANT", (dimension.label, wdaconst)))
                elif constval not in wdaconst:
                    msglist.append(("WDACOLUMNCONSTCHANGED", (dimension.label, constval, wdaconst.pop())))
                headers.remove(dimension.label)
    return headers

def checkvalues(processedrows, headers, wdaseg, msglist):
    oheaders = [OBS]+list(headers)

    # produce counts of each element in case there are duplicates (we are not keeping the orders of the lists)
    ccounts = collections.Counter(tuple(row.get(h)  for h in oheaders)  for row in processedrows)
    wcounts = collections.Counter(tuple(wrow.get(h)  for h in oheaders)  for wrow in wdaseg)
    cset = set(ccounts.keys())
    wset = set(wcounts.keys())
    
    cdiffextra = cset.difference(wset)
    sdiffextra = wset.difference(cset)
    if cdiffextra:
        msglist.append(("NEWVALUESINSEGMENT", cdiffextra))
    if sdiffextra:
        msglist.append(("WDAEXTRAVALUES", sdiffextra))
        
    dupmismatch = { }
    for s in cset.intersection(wset):
        if ccounts[s] != wcounts[s]:
            dupmismatch[s] = (ccounts[s], wcounts[s])
    if dupmismatch:
        msglist.append(("WDADUPLICATESMISMATCH", dupmismatch))
    
        
wdaseg = readtechnicalCSV(Lx)[0]
msglist = [ ]
headers = extraheaderscheck(c1, wdaseg, msglist)
headers = checktheconstantdimensions(c1, headers, wdaseg, msglist)
checkvalues(c1.processedrows, headers, wdaseg, msglist)
for msg, v in msglist:
    warnings.warn(wdamsgstrings[msg] % v)
    
#[dimension.hbagset  for dimension in c1.dimensions]


/home/goatchurch/sensiblecode/lib/python3.5/site-packages/ipykernel/__main__.py:86: UserWarning: Constant column Category value changed Beatles but was in wda file dBeatles


In [10]:
import collections
collections.Counter([2,3,4,4,8])

Counter({2: 1, 3: 1, 4: 2, 8: 1})

In [36]:
additionalheaders = s1pr[0]
template.headermeasurementnumvaluesSet

template.headermeasurementnumvaluesSet

len(s1pr), len(c1pr)
1

1

{-9, -8, -7, -6, -5, -4, -3, -2, -1, 0}